# Implementation of the SABER Key Encapsulation Algorithm

It is a repository with the source code and materials for the project of the 2024 class of the *Cryptography methods in data science* course at the AGH University of Krakow. The scope of the project is to implement the post-quantum [SABER](https://www.esat.kuleuven.be/cosic/pqcrypto/saber/index.html) algorithm in a high-level programming language - Python.

The implementation is based on the SABER specification document, esspecially relevant sections are:
- [Section 2](https://www.esat.kuleuven.be/cosic/pqcrypto/saber/files/saberspecround3.pdf#page=7.37): General algorithm specification,
- [Section 8](https://www.esat.kuleuven.be/cosic/pqcrypto/saber/files/saberspecround3.pdf#page=23.80): Technical Specifications.

## Relevant links

- SABER main page: https://www.esat.kuleuven.be/cosic/pqcrypto/saber/index.html
- Specification PDF (located in `/materials/saberspecround3.pdf`): https://www.esat.kuleuven.be/cosic/pqcrypto/saber/files/saberspecround3.pdf
- Official SABER implementation in C: https://github.com/KULeuven-COSIC/SABER
- Third party C++ implementation: https://github.com/itzmeanjan/saber

## Getting started

In [19]:
# Install the `saber` package:
# !git clone https://github.com/Przemyslaw11/Saber.git
# !cd Saber/
# !pip install -e .

# Required imports
from utils.constants import CONSTANTS_LIGHT_SABER
from utils.algorithms import randombytes
from Crypto.Cipher import AES
from pke import PKE
from kem import KEM

## Example 1: Classic public key encryption (PKE)
In this example Alice (sender) wants to send data to Bob (receiver). The symmetric key is safely distributed thanks to the PKE system. Once the symmetric key is shared, they can communicate securely and efficiently with symmetric encryption, e.g. AES.

### SABER PKE specificaiton:
<img src="../materials/_img/spec-pke.png" width="750">

### Definition of the cryptosystem

In [20]:
# Generate a random seed
seed = randombytes(CONSTANTS_LIGHT_SABER["SABER_SEEDBYTES"])

# Define the PKE class
pke = PKE(**CONSTANTS_LIGHT_SABER)

### Bob generates a public/secret key pair
He sends the public key to Alice, and keeps the secret key to himself.

In [21]:
pk_B, sk_B = pke.KeyGen()

### Alice encrypts the symmetric key with Bob's public key
She generates the symmetric key, encrypts it with Bob's public key, and sends it back to Bob.

In [22]:
# The symmetric key to encrypt
symmetric_key_alice = bytes("This is my awesome symmetric key", encoding="utf-8")

# Encrypt the symmetric key with the Bob's public key
ciphertext = pke.Enc(symmetric_key_alice, seed, pk_B)

### Bob decrypts the symmetric key with his secret key

In [23]:
# Decrypt the message with the secret key
symmetric_key_bob = pke.Dec(ciphertext, sk_B)

### Symmetric key encryption for efficient communication
Now both Alice and Bob have the same symmetric key, and they can use it to communicate efficiently using the symmetric key encryption scheme, for example, AES.

In [24]:
assert symmetric_key_alice == symmetric_key_bob, "Decryption failed!"

iv = randombytes(16)

#### 1. Alice sends data to Bob

In [25]:
aes_A = AES.new(symmetric_key_alice, AES.MODE_CBC, iv)

data = "Hi Bob, it's Alice! Have you heard about the SABER algorithm?".encode('utf-8')
data = data + b"\x00" * (16 - len(data) % 16)  # Padding (if needed)
aes_encrypted= aes_A.encrypt(data)

#### 2. Bob decrypts the data

In [26]:
aes_B = AES.new(symmetric_key_bob, AES.MODE_CBC, iv)

decrypted_data = aes_B.decrypt(aes_encrypted)

#### 3. Test

In [27]:
print(f"Original data:  {data}")
print(f"Decrypted data: {decrypted_data}")

assert data == decrypted_data, "Decryption failed!"
print("Decryption successful!")

Original data:  b"Hi Bob, it's Alice! Have you heard about the SABER algorithm?\x00\x00\x00"
Decrypted data: b"Hi Bob, it's Alice! Have you heard about the SABER algorithm?\x00\x00\x00"
Decryption successful!


## Example 2: Key encapsulation mechanism (KEM)
This use case is similar to the previous one, but thanks to KEM, Alice gets the symetric key along the ciphertext in one step, from Bob's public key. She does not need to generate the symmetric key by herself. Bob can decrypt the recived ciphertext to get the same symmetric key as Alice has and they can communicate securely and efficiently with symmetric encryption, e.g. AES.

### SABER KEM specificaiton:

<img src="../materials/_img/spec-kem.png" width="750">

### Definition of the cryptosystem

In [28]:
# Define the PKE class
kem = KEM(**CONSTANTS_LIGHT_SABER)

### Bob generates a public/secret key pair
He sends the public key to Alice, and keeps the secret key to himself.

In [29]:
pk, sk = kem.KeyGen()

### Alice encapsulates the Bob's public key
She encapsulates the Bob's public key to get the session key along the ciphertext. She keeps the session key and sends the ciphertext to Bob.

In [30]:
session_key_alice, ciphertext = kem.Encaps(pk)

### Eve intercepts the public key
Eve intercepts the public key and tries to get the session key. But her attempt is unsuccessfull because each encapsulation is random.

In [31]:
session_key_eve = kem.Encaps(pk)

assert session_key_alice != session_key_eve, "Eavesdropping Eve has compromised the session key!"
print("Session key is secure!")

Session key is secure!


### Bob decapsulates the ciphertext
He decapsulates the ciphertext with his secret key to get the session key. Now both Alice and Bob have the same symmetric key.

In [32]:
session_key_bob = kem.Decaps(ciphertext, sk)

### Symmetric key encryption for efficient communication
Now both Alice and Bob have the same session key (symmetric), and they can use it to communicate efficiently using the symmetric key encryption scheme, for example, AES.

In [33]:
assert session_key_alice == session_key_bob, "Wrong session key!"

iv = randombytes(16)

#### 1. Alice sends data to Bob

In [34]:
aes_A = AES.new(session_key_alice, AES.MODE_CBC, iv)

data = 'Hi Bob! Our communication is now resilient for quantum atacks!'.encode('utf-8')
data = data + b"\x00" * (16 - len(data) % 16)  # Padding (if needed)
aes_encrypted= aes_A.encrypt(data)

#### 2. Bob decrypts the data

In [35]:
aes_B = AES.new(session_key_bob, AES.MODE_CBC, iv)

decrypted_data = aes_B.decrypt(aes_encrypted)

#### 3. Test

In [36]:
print(f"Original data:  {data}")
print(f"Decrypted data: {decrypted_data}")

assert data == decrypted_data, "Decryption failed!"
print("Decryption successful!")

Original data:  b'Hi Bob! Our communication is now resilient for quantum atacks!\x00\x00'
Decrypted data: b'Hi Bob! Our communication is now resilient for quantum atacks!\x00\x00'
Decryption successful!
